In [1]:
#importing Libraries
import pandas  as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#reading train dataset
train = pd.read_csv('train.csv')

In [3]:
train.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [4]:
train.columns

Index(['id', 'original_text', 'lang', 'retweet_count', 'original_author',
       'sentiment_class'],
      dtype='object')

In [5]:
train.shape

(3235, 6)

In [0]:
#cleaning data *Removing unwanted text, numbers, special chars etc., * change to lower case * spliting the words *appling stemming * removing stopwords 
train_clean = []
for i in range(0, 3235):
  review = re.sub('[^a-zA-Z]',' ',train['original_text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review =' '.join(review)
  train_clean.append(review)

In [7]:
train_clean[0:10]

['happi mothersday amaz mother know hard abl see mother today us protect vulner member societi beatcoronaviru pic twitter com va nfjfq b',
 'happi mother day mum sorri bring mother day flower cwtch honestli point walk hot coal abl bell soon love lot xxx p need photo http photo app goo gl vxblrsczd te',
 'happi mother day mother day work today quiet time reflect dog walk finish jigsaw garden learn guitar chord drunk strawberri gin tonic watch lee even dvd favourit place visit isol pic twitter com gz xvvf f',
 'happi mother day beauti woman royalti sooth mummi jeremi emerald prayforroksi ultimateloveng pic twitter com oeeti pvv',
 'rememb amaz ladi made late grandmoth iri mum carol great grandmoth ethel miss never forgotten happi mother day great mum love sent xxxx pic twitter com xzzzdeybj',
 'happi mother day bratz world familiez yasmin mum portia bratz bratzworldfamiliez bratz bratzkidz bratzyasmin bratzportia theonlygirlswithapassionforfashion bratzdol bratzcollector happymothersday 

In [8]:
#bagging words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=7319)
X= cv.fit_transform(train_clean).toarray()
y= train.iloc[: , 5].values
y

array([ 0,  0, -1, ...,  0,  0, -1])

In [9]:
X.shape

(3235, 7319)

In [0]:
#reading test dataset
test = pd.read_csv('test.csv')

In [11]:
test.head()

,id,original_text,lang,retweet_count,original_author
0,1.246628e+18,"3. Yeah, I once cooked potatoes when I was 3 y...",en,0,LToddWood
1,1.245898e+18,"Happy Mother's Day to all the mums, step-mums,...",en,0,iiarushii
2,1.244717e+18,"I love the people from the UK, however, when I...",en,0,andreaanderegg
3,1.245730e+18,Happy 81st Birthday Happy Mother’s Day to my m...,en,1,TheBookTweeters
4,1.244636e+18,Happy Mothers day to all those wonderful mothe...,en,0,andreaanderegg


In [12]:
test.shape

(1387, 5)

In [0]:
test_clean = []
for i in range(0, 1387):
  review = re.sub('[^a-zA-Z]',' ',test['original_text'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review =' '.join(review)
  test_clean.append(review)

In [14]:
print(len(test_clean))

1387


In [15]:
test_cv = CountVectorizer()
test_count= cv.fit_transform(test_clean).toarray()
test_count

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
test_count.shape

(1387, 7319)

In [17]:
#spliting the dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
train.columns

Index(['id', 'original_text', 'lang', 'retweet_count', 'original_author',
       'sentiment_class'],
      dtype='object')

In [18]:
#model building RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
 #predicting tweets
 ran_predict = classifier.predict(X_test)

In [20]:
#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, ran_predict)
cm

array([[  3, 196,   1],
       [  7, 387,   8],
       [  0, 202,   5]])

In [21]:
from sklearn.metrics import classification_report
cr = classification_report(y_test,ran_predict, digits=3)
print(cr)

              precision    recall  f1-score   support

          -1      0.300     0.015     0.029       200
           0      0.493     0.963     0.652       402
           1      0.357     0.024     0.045       207

    accuracy                          0.488       809
   macro avg      0.383     0.334     0.242       809
weighted avg      0.411     0.488     0.343       809



In [22]:
#model building XGBClassifier
from xgboost import XGBClassifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
#prediction
xgb_predict = xgb_classifier.predict(X_test)

In [24]:
#Confusion matrix XG Boost
xgb_cm = confusion_matrix(y_test, xgb_predict)
xgb_cm

array([[  3, 197,   0],
       [  2, 396,   4],
       [  0, 204,   3]])

In [25]:
xgb_score = classification_report(y_test,xgb_predict)
print(xgb_score)

              precision    recall  f1-score   support

          -1       0.60      0.01      0.03       200
           0       0.50      0.99      0.66       402
           1       0.43      0.01      0.03       207

    accuracy                           0.50       809
   macro avg       0.51      0.34      0.24       809
weighted avg       0.50      0.50      0.34       809



In [26]:
#model building Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
gbc_classifier= GradientBoostingClassifier(n_estimators= 280)
gbc_classifier.fit(X_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=280,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
gbc_predict = gbc_classifier.predict(X_test)

In [28]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
gbc = confusion_matrix(y_test, gbc_predict)
gbc

array([[ 11, 186,   3],
       [ 15, 373,  14],
       [  9, 188,  10]])

In [29]:
from sklearn.metrics import classification_report
gbc_score = classification_report(y_test,gbc_predict)
print(gbc_score)

              precision    recall  f1-score   support

          -1       0.31      0.06      0.09       200
           0       0.50      0.93      0.65       402
           1       0.37      0.05      0.09       207

    accuracy                           0.49       809
   macro avg       0.39      0.34      0.28       809
weighted avg       0.42      0.49      0.37       809



In [0]:
review_id = test['id']
predict_values = gbc_classifier.predict(test_count)

output = pd.DataFrame({ 'id' : review_id, 'sentiment_class': predict_values })
output.to_csv('submission.csv', index=False)